# Experiment Notebook: System Metrics

# Table of Contents
* [Experiment Summary](#Experiment-Summary)
* [Experiment Assumptions](#Experiment-Assumptions)
* [Experiment Setup](#Experiment-Setup)
* [Analysis 1: Sanity Checks](#Analysis-1:-Sanity-Checks)
* [Analysis 2: Correlation Matrix](#Analysis-1:-Correlation-Matrix)

# Experiment Summary 

The purpose of this notebook is to demonstrate the system's standard metrics, KPIs and goals.

# Experiment Assumptions

See [assumptions document](../../ASSUMPTIONS.md) for further details.

# Experiment Setup

We begin with several experiment-notebook-level preparatory setup operations:

* Import relevant dependencies
* Import relevant experiment templates
* Create copies of experiments
* Configure and customize experiments 

Analysis-specific setup operations are handled in their respective notebook sections.

In [ ]:
# Import the setup module:
# * sets up the Python path
# * runs shared notebook configuration methods, such as loading IPython modules
import setup

import copy
import logging
import numpy as np
import pandas as pd
import plotly.express as px

import experiments.notebooks.visualizations as visualizations
from experiments.run import run
from experiments.utils import display_code

In [ ]:
# Enable/disable logging
logger = logging.getLogger()
logger.disabled = False

In [ ]:
# Import experiment templates
import experiments.default_experiment as default_experiment

In [ ]:
# Inspect experiment template
display_code(default_experiment)

In [ ]:
# Create a simulation for each analysis
simulation_1 = copy.deepcopy(default_experiment.experiment.simulations[0])
simulation_2 = copy.deepcopy(default_experiment.experiment.simulations[0])
simulation_3 = copy.deepcopy(default_experiment.experiment.simulations[0])

In [ ]:
# Experiment configuration
# simulation_1.model.initial_state.update({})
# simulation_1.model.params.update({})

# Analysis 1: PCV Sanity Checks

A simulation across 4 volatile asset price scenarios to validate PCV states and metrics.

In [ ]:
# Analysis-specific setup
simulation_1.model.params.update({
    "volatile_asset_price_process": [
        lambda _run, _timestep: 2_000,
        lambda _run, timestep: 2_000 if timestep < 365 / 4 else (1_000 if timestep < 365 * 3/4 else 2_000),
        lambda _run, timestep: 2_000 * (1 + timestep * 0.2 / 365),
        lambda _run, timestep: 2_000 * (1 - timestep * 0.2 / 365),
    ],
})

In [ ]:
# Experiment execution
df, exceptions = run(simulation_1)

In [ ]:
# Post-processing and visualizations

In [ ]:
fig = df.plot(y='volatile_asset_price', color='subset')

fig.update_xaxes(title='Timestamp')

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import itertools
from experiments.notebooks.visualizations.plotly_theme import cadlabs_colorway_sequence
color_cycle = itertools.cycle(cadlabs_colorway_sequence)


fig = make_subplots(rows=5, cols=len(df.subset.unique()), shared_yaxes=True)

for subset in df.subset.unique():
    df_plot = df.query('subset == @subset')
    
    fig.add_trace(
        go.Scatter(
            x=df_plot.timestamp,
            y=df_plot.volatile_asset_price,
            name="Volatile Asset Price",
            line=dict(color=cadlabs_colorway_sequence[0]),
            showlegend=(True if subset == 0 else False),
        ),
        row=1, col=subset+1,
    )

    fig.add_trace(
        go.Scatter(
            x=df_plot.timestamp,
            y=df_plot.total_pcv,
            name="Total PCV",
            line=dict(color=cadlabs_colorway_sequence[1]),
            showlegend=(True if subset == 0 else False),
        ),
        row=2, col=subset+1
    )

    fig.add_trace(
        go.Scatter(
            x=df_plot.timestamp,
            y=df_plot.collateralization_ratio,
            name="Collateralization Ratio",
            line=dict(color=cadlabs_colorway_sequence[2]),
            showlegend=(True if subset == 0 else False),
        ),
        row=3, col=subset+1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df_plot.timestamp,
            y=df_plot.total_volatile_asset_pcv,
            name="Volatile Asset PCV",
            line=dict(color=cadlabs_colorway_sequence[3]),
            showlegend=(True if subset == 0 else False),
            stackgroup='one',
        ),
        row=4, col=subset+1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df_plot.timestamp,
            y=df_plot.total_stable_asset_pcv,
            name="Stable Asset PCV",
            line=dict(color=cadlabs_colorway_sequence[4]),
            showlegend=(True if subset == 0 else False),
            stackgroup='one',
        ),
        row=4, col=subset+1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df_plot.timestamp,
            y=df_plot.liquidity_pool_tvl,
            name="Liquidity Pool TVL",
            line=dict(color=cadlabs_colorway_sequence[4]),
            showlegend=(True if subset == 0 else False),
        ),
        row=5, col=subset+1
    )


fig.update_layout(height=1000, title_text="PCV Sanity Checks")
fig.show()

# Analysis 2: Correlation Matrix

In [ ]:
# Analysis-specific setup
simulation_2.model.params.update({})

In [ ]:
# Experiment execution
df, exceptions = run(simulation_2)

In [ ]:
# Post-processing and visualizations

In [ ]:
# import seaborn as sns

# corr = df[[
#     "volatile_asset_price",
#     "total_pcv",
#     "collateralization_ratio",
#     "total_stable_asset_pcv",
#     "liquidity_pool_tvl"
# ]].corr()

# _ = sns.heatmap(corr,
#             xticklabels=corr.columns.values,
#             yticklabels=corr.columns.values)

# import plotly.plotly as py
# from plotly.graph_objs import *

# trace1 = {
#   "type": "heatmap", 
#   "x": ["fp_time", "uid", "revenue", "num_trans", "act_days", "lt", "delt_pmnt_log"], 
#   "y": ["fp_time", "uid", "revenue", "num_trans", "act_days", "lt", "delt_pmnt_log"], 
#   "z": [],
# }

# data = Data([trace1])
# layout = {"title": "Features Correlation Matrix"}
# fig = Figure(data=data, layout=layout)
# plot_url = py.plot(fig)

# PCV at Risk

In [ ]:
from operator import lt, gt

In [ ]:
simulation_3.runs = 20

parameter_overrides = {
    "target_rebalancing_condition": [gt, lt], # Simulate decrease and increase of stable PCV
    "target_stable_pcv_ratio": [0.2, 0.5], # Simulate decrease and increase of stable PCV
    "rebalancing_period": [int(365 / 4)],
}

simulation_3.model.params.update(parameter_overrides)

In [ ]:
# Experiment execution
df, exceptions = run(simulation_3)

In [ ]:
df

In [ ]:
df.plot(y="volatile_asset_price", color="run", facet_row="subset")

In [ ]:
df.plot(y="total_pcv", color="run", facet_row="subset")

## vectorized VAR

In [ ]:
def calculate_VaR_run(df, n_run, alpha, n_timesteps, t_start, t_end):
    pcv_ret = df.query('run==@n_run and (timestep > @t_start and timestep <= @t_end)')['total_pcv'].pct_change()
    pcv_final_val = df.query('run==@n_run')['total_pcv'].iloc[-1]
    q = pcv_ret.quantile(1-alpha)
    # see https://www0.gsb.columbia.edu/faculty/pglasserman/B6014/var-d.pdf
    # for n-day var simplifying assumption which allows for generalization
    VaR_n = abs(pcv_final_val * q)*np.sqrt(n_timesteps)
    
    return VaR_n, q

In [ ]:
def calculate_VaR_subset(df, n_subset, alpha, n_timesteps, t_start, t_end):
    
    VAR = []
    
    df_ = df.query("subset==@n_subset")
    for run in df_['run'].value_counts().index:
        var, q = calculate_VaR_run(df_, run, alpha, n_timesteps, t_start, t_end)
        
        VAR.append((n_subset, var, q))
    
    return pd.DataFrame(VAR, columns=[x+'_'+str(n_timesteps) for x in ['subset', 'VaR', 'q']])

In [ ]:
def calculate_VaR(df, alpha, n_timesteps, t_start, t_end):
    
    L = []    
    for subset in df['subset'].value_counts().index:
        VaR_subset = calculate_VaR_subset(df, subset, alpha, n_timesteps, t_start, t_end)
        L.append(VaR_subset)
        
    return pd.concat(L, axis=0).reset_index(drop=True)

In [ ]:
def calculate_VaR_n(df, alpha, timestep_range, t_start, t_end):
    U, L = [], []
    
    for t in range(timestep_range):
        L.append(calculate_VaR(df, 0.95, t+1, t_start, t_end))
        U.append(t+1)
        
    return dict(zip(U, L))

In [ ]:
def calculate_VaR_summary_stats(df, n_timesteps):
    L = []
    colnames = []
    for subset in df['subset'+'_'+str(n_timesteps)].value_counts().index:
        L.append(df.query('subset'+'_'+str(n_timesteps)+'==@subset').describe())
        colnames += [colname+'_'+str(subset) for colname in df.columns]
    
    VAR_info = pd.concat(L, axis=1)
    VAR_info.columns = colnames
    VAR_info = VAR_info.drop(index=['count'])
    return VAR_info
        

In [ ]:
# calculate 1-day vectorized VaR for all simulation outputs
# set window bounds, whole simulation for simplicity
t_start = 0
t_end = 365
alpha = 0.95
max_day_VaR = 10
VAR_df = calculate_VaR_n(df, alpha, max_day_VaR, t_start, t_end)

# NOTE: create rolling window by further vectorizing VaR calculation by iterating over start and end time

In [ ]:
VAR_1_stats = calculate_VaR_summary_stats(VAR_df[1], 1)

In [ ]:
VAR_10_stats = calculate_VaR_summary_stats(VAR_df[10], 10)

In [ ]:
# look at 1 day VaR

In [ ]:
VAR_1_stats

In [ ]:
# TODO: iterate the print statement

In [ ]:
print('Over the simulation, for parameters in the sweep corresponding to subset 0,',
      np.round(VAR_1_stats['VaR_1_0'].loc['mean'], 2), 'USD is the mean 1-Day PCVaR at 95% over the 20 runs performed, with associated',
      np.round(VAR_1_stats['q_1_0'].loc['mean'], 4), '% average 5% quantile percentile loss')

In [ ]:
print('Over the simulation, for parameters in the sweep corresponding to subset 1,',
      np.round(VAR_1_stats['VaR_1_1'].loc['mean'], 2), 'USD is the mean 1-Day PCVaR at 95% over the 20 runs performed, with associated',
      np.round(VAR_1_stats['q_1_1'].loc['mean'], 4), '% average 5% quantile percentile loss')

In [ ]:
# look at 10 day VaR

In [ ]:
VAR_10_stats

In [ ]:
print('Over the simulation, for parameters in the sweep corresponding to subset 0,',
      np.round(VAR_10_stats['VaR_10_0'].loc['mean'], 2), 'USD is the mean 10-Day PCVaR at 95% over the 20 runs performed, with associated',
      np.round(VAR_10_stats['q_10_0'].loc['mean'], 4), '% average 5% quantile percentile loss')

In [ ]:
print('Over the simulation, for parameters in the sweep corresponding to subset 1,',
      np.round(VAR_10_stats['VaR_10_1'].loc['mean'], 2), 'USD is the mean 10-Day PCVaR at 95% over the 20 runs performed, with associated',
      np.round(VAR_10_stats['q_10_1'].loc['mean'], 4), '% average 5% quantile percentile loss')

## generic state variable summary stats

In [ ]:
def get_summary_stats_for_simulation_average(df, subset, cols=None):
    
    df_ = df[cols].groupby(['subset','timestep']).mean().reset_index().query('subset==@subset')
    stats_df = df_.describe()
    stats_df.loc['skew'] = df_.skew()
    stats_df.loc['kurtosis'] = df_.kurtosis()
    # TODO: max drawdown & other relevant summary stats here

    return stats_df

In [ ]:
def get_summary_stats_for_simulation_run(df, subset, run, cols=None):
    df_ = df[cols].query('subset==@subset and run==@run')

    stats_df = df_.describe()
    stats_df.loc['skew'] = df_.skew()
    stats_df.loc['kurtosis'] = df_.kurtosis()
    # TODO: max drawdown & other relevant summary stats here

    return stats_df

In [ ]:
# pass only subset of relevant state variables here
get_summary_stats_for_simulation_average(df, 0, cols=df.columns)

In [ ]:
get_summary_stats_for_simulation_run(df, 0, 1, cols=df.columns)

## capital allocation metrics

In [ ]:
def normalize_fei_deposits(df, fei_cols):
    
    df_addtl_cols = ['subset', 'run', 'timestep']
    
    norm_df = df[fei_cols].div(df[fei_cols].sum(axis=1), axis=0)
    
    return pd.concat([df[df_addtl_cols], norm_df], axis=1)

In [ ]:
def get_fei_deposit_colnames(col_prefixes, col_suffix):
    return [x+col_suffix for x in col_prefixes]

In [ ]:
def get_allocations_at_timestep(fei_capital_allocations, fei_cols, run, subset, timestep=-1):
    df_ = fei_capital_allocations.query('run==@run and subset==@subset')[fei_cols].iloc[timestep]
    df_ = pd.DataFrame(df_).reset_index()
    df_.columns = ['index', 'values']
    
    return df_

In [ ]:
def get_avg_allocations_at_timestep(fei_capital_allocations, fei_cols, subset, timestep=-1):
    df_ = fei_capital_allocations.query('subset==@subset')[fei_cols].iloc[timestep]
    df_ = pd.DataFrame(df_).reset_index()
    df_.columns = ['index', 'values']
    
    return df_

In [ ]:
fei_deposit_prefixes = ['fei_deposit_idle','fei_deposit_liquidity_pool', 'fei_deposit_money_market']

In [ ]:
fei_deposit_balances = get_fei_deposit_colnames(fei_deposit_prefixes, '_balance')

In [ ]:
fei_balance_capital_allocations = normalize_fei_deposits(df, fei_deposit_balances)

In [ ]:
fei_balance_capital_allocations

In [ ]:
# For one run where sim is driven by stochastic processes

In [ ]:
fei_balance_capital_allocations.query('run==1').plot(y=fei_deposit_balances, facet_row="subset")

In [ ]:
timestep_allocations_0 = get_allocations_at_timestep(fei_balance_capital_allocations, fei_deposit_balances, 1, 0)

In [ ]:
timestep_allocations_1 = get_allocations_at_timestep(fei_balance_capital_allocations, fei_deposit_balances, 1, 0)

In [ ]:
fig = px.pie(timestep_allocations_0, values='values', names='index', title='FEI Ecosystem Capital Allocation for Subset 0, run 1')
fig.show()

In [ ]:
fig = px.pie(timestep_allocations_1, values='values', names='index', title='FEI Ecosystem Capital Allocation for Subset 1, run 1')
fig.show()

In [ ]:
# Average over simulation runs

In [ ]:
def get_average_capital_allocations(fei_capital_allocations):
    df_ = fei_balance_capital_allocations.groupby(['subset','timestep']).mean().reset_index()
    df_ = df_.drop(columns='run')
    return df_

In [ ]:
fei_balance_avg_capital_allocations = get_average_capital_allocations(fei_balance_capital_allocations)

In [ ]:
avg_timestep_allocations_0 = get_avg_allocations_at_timestep(fei_balance_avg_capital_allocations, fei_deposit_balances, 0)

In [ ]:
avg_timestep_allocations_1 = get_avg_allocations_at_timestep(fei_balance_avg_capital_allocations, fei_deposit_balances, 1)

In [ ]:
fig = px.pie(avg_timestep_allocations_0, values='values', names='index', title='FEI Ecosystem Capital Allocation for Subset 0, averaged over all runs')
fig.show()

In [ ]:
fig = px.pie(avg_timestep_allocations_1, values='values', names='index', title='FEI Ecosystem Capital Allocation for Subset 1, averaged over all runs')
fig.show()